In [ ]:
!gdown 12WhRAutMZ_PKEWAYyxKk6MJ_hZ9t6g_q
#https://drive.google.com/file/d/12WhRAutMZ_PKEWAYyxKk6MJ_hZ9t6g_q/view?usp=drive_link

In [ ]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

id2label = {0: 'A',
            1: 'B',
            2: 'C',
            3: 'D',
            4: 'E'}
label2id = {'A': 0,
            'B': 1,
            'C': 2,
            'D': 3,
            'E': 4}
num_labels = len(id2label)

model_name = "allenai/scibert_scivocab_uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          num_labels=num_labels,
                                          label2id=label2id,
                                          id2label=id2label,
                                          use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=num_labels)

# peft_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.05, target_modules=["query", "vavlue"], bias="none")
# model = get_peft_model(model, peft_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
model.load_state_dict(torch.load("./pytorch_model.bin", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
convert_to_binary = {0: '10000',
                     1: '01000',
                     2: '00100',
                     3: '00010',
                     4: '00001'}

def predict_per_sample(df, model, tokenizer):
    answers = []
    for idx, row in df.iterrows():
        context = row["retrieved_context"]
        question = row["question"]
        choices = [op for op in row[["option_1", "option_2", "option_3", "option_4", "option_5", "option_6"]].tolist() if isinstance(op, str)]
        text_choices = " ".join(choices)
        prompt = f"Question: {question}. Choice the correct answers from: {text_choices}. Context: {context}"
        model_inputs = tokenizer(prompt, max_length = 512, truncation=True, return_tensors="pt")
        outputs = model(**model_inputs)
        prediction = torch.argmax(outputs[0], axis=1)
        answer = convert_to_binary[prediction.item()]
        answer = answer[:len(choices)]
        answers.append(answer)
    return answers

In [ ]:
import pandas as pd
df = pd.read_csv("/content/updated_en_test_data_v2.csv") #testset
answers = predict_per_sample(df, model, tokenizer)
df['answer'] = answers
result_df = df[["id", "answer"]]
result_df.to_csv("/content/updated_answer_en_test_data_v2.csv")